### Data Wrangling Mini Project

I am a huge football fan, so I have chosen a football data set for this demonstration.

#### The goal (of the mini project)
My goal is to analyze the performance of the Peruvian national team in World Cup qualifying games since France 1998. For the France 1998, Conmebol (the South American Football federation) updated the format of the world cup qualifiers and decided all teams should face each other during the qualifiers. It is the only federation with this format.

But before we can start analyzing the performance of Peru, we need to do some data wrangling and cleaning. 

#### The dataset
Let's discuss the data we have available, and then I'll cover what I want as the final data set (remember this is a data wrangling and preparation project).

From Kaggle: "This dataset includes 44,060 results of international football matches starting from the very first official match in 1972 up to 2019. The matches range from FIFA World Cup to FIFI Wild Cup to regular friendly matches."

The data includes the following columns:

* **_date_** - date of the match
* **_home_team_** - the name of the home team
* **_away_team_** - the name of the away team
* **_home_score_** - full-time home team score including extra time, not including penalty-shootouts
* **_away_score_** - full-time away team score including extra time, not including penalty-shootouts
* **_tournament_** - the name of the tournament
* **_city_** - the name of the city/town/administrative unit where the match was played
* **_country_** - the name of the country where the match was played
* **_neutral_** - TRUE/FALSE column indicating whether the match was played at a neutral venue

#### The approach
To prepare the smaller data set with all the games played by the Peruvian national team, we'll do the following:

1. Filter out games that do not involve Peru
2. Filter out games that are not world cup qualifiers
3. Filter out games that were before 1996 (when France 1998 qualifiers began)
4. Filter out games against New Zewland (part of intercontinental playoffs)
5. Add points relative to wins, draws, lossess
6. Add year, month, and day dimensions
7. Add goal difference
8. Add the world cup qualification process to later (in a separate notebook) do analysis
9. Add home game flag

#### The final dataset

The final dataset will look as follows:

* **_date_** - date of the match
* **_world_cup_** - the world cup process
* **_year_** - year dimension
* **_month_** - month dimension
* **_day_** - day dimensinos
* **_is_home_game_** - home game flag
* **_main_team_** - the main team of interest
* **_rival_team_** - the rival team
* **_win_loss_** - win/loss/draw flag
* **_points_** - points for result obtained
* **_goals_favor_** - goals scored
* **_goals_against_** - goals conceded
* **_goal_diff_** - goals scored minues goals conceded

Let's dive into it!

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
df = pd.read_csv('results.csv')

Let's take a quick peek at the data.

In [3]:
df.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


A few things to notice:
* **_date_** is not in date or datetime format. It's in string format.
* **_home_score_** has 1 null value
* **_away_score_** has 1 null value

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44060 entries, 0 to 44059
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        44060 non-null  object 
 1   home_team   44060 non-null  object 
 2   away_team   44060 non-null  object 
 3   home_score  44059 non-null  float64
 4   away_score  44059 non-null  float64
 5   tournament  44060 non-null  object 
 6   city        44060 non-null  object 
 7   country     44060 non-null  object 
 8   neutral     44060 non-null  bool   
dtypes: bool(1), float64(2), object(6)
memory usage: 2.7+ MB


Luckily the null values in **_home_score_** and **_away_score_** do not involve Peru.

In [5]:
df[(df['home_score'].isnull()==True) | (df['away_score'].isnull()==True)]

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
44059,2022-09-30,Fiji,Solomon Islands,NaN,NaN,MSG Prime Minister's Cup,Luganville,Vanuatu,True


#### Converting the date column to datetime format

In [6]:
df['date'] = pd.to_datetime(df['date'])

#### Filtering out undesired rows/records
Now that we know what type of rows we want to keep, we apply those filters
* Keep games that involve Peru
* Keep games that are world cup qualifiers
* Keep games starting April 24th, 1996 (when France 1998 qualifiers began) and ending before April 1st, 2022 (when the Qatar 2022 qualifiers end)

In [7]:
peru_df = df[
    ((df['home_team'] == 'Peru') | (df['away_team'] == 'Peru'))
    & (df['tournament'] == 'FIFA World Cup qualification')
    & (df['date'] > '1996-01-01')
    & (df['date'] < '2022-04-01')
]

Let's take another peek at the data. 
* We see that all the rows have Peru as the home or away team. 
* All the rows are WC qualifier games.
* Wait, the indices look weird. Let's fix that!

In [8]:
peru_df.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
19819,1996-04-24,Ecuador,Peru,4.0,1.0,FIFA World Cup qualification,Guayaquil,Ecuador,False
19945,1996-06-02,Peru,Colombia,1.0,1.0,FIFA World Cup qualification,Lima,Peru,False
20069,1996-07-07,Peru,Argentina,0.0,0.0,FIFA World Cup qualification,Lima,Peru,False
20155,1996-09-01,Bolivia,Peru,0.0,0.0,FIFA World Cup qualification,La Paz,Bolivia,False
20358,1996-11-10,Peru,Venezuela,4.0,1.0,FIFA World Cup qualification,Lima,Peru,False


In [9]:
peru_df.reset_index(drop=True, inplace=True)

Now it looks better!

In [10]:
peru_df.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1996-04-24,Ecuador,Peru,4.0,1.0,FIFA World Cup qualification,Guayaquil,Ecuador,False
1,1996-06-02,Peru,Colombia,1.0,1.0,FIFA World Cup qualification,Lima,Peru,False
2,1996-07-07,Peru,Argentina,0.0,0.0,FIFA World Cup qualification,Lima,Peru,False
3,1996-09-01,Bolivia,Peru,0.0,0.0,FIFA World Cup qualification,La Paz,Bolivia,False
4,1996-11-10,Peru,Venezuela,4.0,1.0,FIFA World Cup qualification,Lima,Peru,False


In [11]:
peru_df['is_home_game'] = np.where(peru_df['country'] == 'Peru', 1, 0)

In [12]:
peru_df['year'] = peru_df['date'].dt.year
peru_df['month'] = peru_df['date'].dt.month
peru_df['day'] = peru_df['date'].dt.day

In [13]:
def points(row):
    if row['home_score'] == row['away_score']:
        val = 1
    elif row['is_home_game'] == 1 and row['home_score'] > row['away_score']:
        val = 3
    elif row['is_home_game'] == 0 and row['home_score'] < row['away_score']:
        val = 3
    elif row['is_home_game'] == 1 and row['home_score'] < row['away_score']:
        val = 0
    elif row['is_home_game'] == 0 and row['home_score'] > row['away_score']:
        val = 0
    else:
        val = False
    return val

In [14]:
def goal_diff(row):
    if row['is_home_game'] == 1:
        val = row['home_score'] - row['away_score']
    elif row['is_home_game'] == 0:
        val = row['away_score'] - row['home_score']
    else:
        val = False
    return val

In [15]:
def rival(row):
    if row['is_home_game'] == 1:
        val = row['away_team']
    elif row['is_home_game'] == 0:
        val = row['home_team']
    else:
        val = False
    return val

In [16]:
def goal_favor(row):
    if row['is_home_game'] == 1:
        val = row['home_score']
    else:
        val = row['away_score']
    return val

def goal_against(row):
    if row['is_home_game'] == 0:
        val = row['home_score']
    else:
        val = row['away_score']
    return val

In [17]:
def win_loss_tie(row):
    if row['home_score'] == row['away_score']:
        val = 'T'
    elif row['is_home_game'] == 1 and row['home_score'] > row['away_score']:
        val = 'W'
    elif row['is_home_game'] == 0 and row['home_score'] < row['away_score']:
        val = 'W'
    elif row['is_home_game'] == 1 and row['home_score'] < row['away_score']:
        val = 'L'
    elif row['is_home_game'] == 0 and row['home_score'] > row['away_score']:
        val = 'L'
    else:
        val = False
    return val

In [18]:
def wc_year(row):
    if row['year'] == 1996 or row['year'] == 1997:
        val = '1998 France'
    elif row['year'] == 2000 or row['year'] == 2001:
        val = '2002 Japan Korea'
    elif row['year'] >= 2003 and row['year'] <= 2005:
        val = '2006 Germany'
    elif row['year'] >= 2007 and row['year'] <= 2009:
        val = '2010 South Africa'
    elif row['year'] >= 2011 and row['year'] <= 2013:
        val = '2014 Brazil'
    elif row['year'] >= 2015 and row['year'] <= 2017:
        val = '2018 Russia'
    elif row['year'] >= 2020 and row['year'] <= 2022:
        val = '2022 Qatar'
    return val

In [19]:
peru_df['points'] = peru_df.apply(points, axis=1)
peru_df['goal_diff'] = peru_df.apply(goal_diff, axis=1)
peru_df['rival_team'] = peru_df.apply(rival, axis=1)
peru_df['win_loss'] = peru_df.apply(win_loss_tie, axis=1)
peru_df['world_cup'] = peru_df.apply(wc_year, axis=1)
peru_df['goal_favor'] = peru_df.apply(goal_favor, axis=1)
peru_df['goal_against'] = peru_df.apply(goal_against, axis=1)
peru_df['main_team'] = 'Peru'

In [20]:
peru_df = peru_df[peru_df['rival_team'] != 'New Zealand']

In [21]:
peru_df.columns

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral', 'is_home_game', 'year',
       'month', 'day', 'points', 'goal_diff', 'rival_team', 'win_loss',
       'world_cup', 'goal_favor', 'goal_against', 'main_team'],
      dtype='object')

In [22]:
peru_df = peru_df[['date', 'world_cup', 'year', 'month', 'day', 
                   'is_home_game', 'main_team', 'rival_team', 
                   'win_loss', 'points', 'goal_favor', 'goal_against',
                   'goal_diff']]

In [23]:
peru_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122 entries, 0 to 123
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          122 non-null    datetime64[ns]
 1   world_cup     122 non-null    object        
 2   year          122 non-null    int64         
 3   month         122 non-null    int64         
 4   day           122 non-null    int64         
 5   is_home_game  122 non-null    int32         
 6   main_team     122 non-null    object        
 7   rival_team    122 non-null    object        
 8   win_loss      122 non-null    object        
 9   points        122 non-null    int64         
 10  goal_favor    122 non-null    float64       
 11  goal_against  122 non-null    float64       
 12  goal_diff     122 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int32(1), int64(4), object(4)
memory usage: 12.9+ KB


In [24]:
peru_df.head(5)

,date,world_cup,year,month,day,is_home_game,main_team,rival_team,win_loss,points,goal_favor,goal_against,goal_diff
0,1996-04-24,1998 France,1996,4,24,0,Peru,Ecuador,L,0,1.0,4.0,-3.0
1,1996-06-02,1998 France,1996,6,2,1,Peru,Colombia,T,1,1.0,1.0,0.0
2,1996-07-07,1998 France,1996,7,7,1,Peru,Argentina,T,1,0.0,0.0,0.0
3,1996-09-01,1998 France,1996,9,1,0,Peru,Bolivia,T,1,0.0,0.0,0.0
4,1996-11-10,1998 France,1996,11,10,1,Peru,Venezuela,W,3,4.0,1.0,3.0


Writing the resulting dataset to a csv file. We'll use it in the next notebooks.

In [25]:
peru_df.to_csv('peru_wc_results.csv', index = False)